In [6]:
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.by import By
import numpy as np

In [9]:
def Crawling(driver, page, type, url_xpath, df_alchol, i):
    url = 'https://business.veluga.kr/search/result/?page=' + str(page) + '&drink_type_id=' + str(type)
    driver.get(url)
    click = driver.find_element(By.XPATH, url_xpath)
    driver.execute_script("arguments[0].click();", click)
    time.sleep(3)
    image_element = driver.find_elements(By.XPATH,'//*[@id="__next"]/div/div/div[1]/img')[0]
    image_td = image_element.get_attribute('src')

    name =  driver.find_element(By.XPATH,'//*[@id="__next"]/div/div/div[2]/div[1]/h1').text
    
    percent =  driver.find_element(By.XPATH,"//th[contains(text(), '도수')]")
    percent_td = percent.find_element(By.XPATH,"following-sibling::td").text

    category =  driver.find_element(By.XPATH,"//th[contains(text(), '스타일')]")
    category_td = category.find_element(By.XPATH,"following-sibling::td").text

    produce =  driver.find_element(By.XPATH,"//th[contains(text(), '제조사')]")
    produce_td = produce.find_element(By.XPATH,"following-sibling::td").text

    supply = driver.find_element(By.XPATH,"//th[contains(text(), '공급사')]")
    supply_td = supply.find_element(By.XPATH,"following-sibling::td").text

    index = index + 1

    df_alchol.loc[index] = [name, percent_td, category_td, produce_td, supply_td, image_td]
    i+1
    pass

In [14]:
from selenium.common.exceptions import ElementNotInteractableException, ElementClickInterceptedException, NoSuchElementException

df_alchol = pd.DataFrame(columns = ['name', 'percent', 'category', 'produce', 'supply', 'img_url'])
driver = webdriver.Chrome(executable_path='chromedriver')
index = 0
for page in range(1, 7+1):
    ## 크롤링할 페이지 변수 
    type = 13
    url = 'https://business.veluga.kr/search/result/?page=' + str(page) + '&drink_type_id=' + str(type)
    driver.get(url)

    print("현재 " + str(page) + "페이지 크롤링 중")
 
    # 페이지의 총 20개 아이템을 하나씩 돌면서 크롤링
    for i in range(1, 20 + 1):
        
        time.sleep(3)
        base_xpath = '//*[@id="__next"]/div/div/div[1]/a['
        tail_xpath = ']/div[1]/img'
        url_xpath = base_xpath + str(i) + tail_xpath
        
        try: 
            click = driver.find_element(By.XPATH, url_xpath)
            driver.execute_script("arguments[0].click();", click)
            time.sleep(3)

            # 이미지 url 
            image_element = driver.find_elements(By.XPATH,'//*[@id="__next"]/div/div/div[1]/img')[0]
            image_td = image_element.get_attribute('src')

            # 주류 이름
            name =  driver.find_element(By.XPATH,'//*[@id="__next"]/div/div/div[2]/div[1]/h1').text
            
            # 주류 도수
            percent =  driver.find_element(By.XPATH,"//th[contains(text(), '도수')]")
            percent_td = percent.find_element(By.XPATH,"following-sibling::td").text

            # 주류 카테고리
            category =  driver.find_element(By.XPATH,"//th[contains(text(), '스타일')]")
            category_td = category.find_element(By.XPATH,"following-sibling::td").text

            # 주류 제조사
            produce =  driver.find_element(By.XPATH,"//th[contains(text(), '제조사')]")
            produce_td = produce.find_element(By.XPATH,"following-sibling::td").text
            # 주류 공급사
            supply = driver.find_element(By.XPATH,"//th[contains(text(), '공급사')]")
            supply_td = supply.find_element(By.XPATH,"following-sibling::td").text

            # 크롤링한 데이터 데이터프레임에 넣기
            df_alchol.loc[index] = [name, percent_td, category_td, produce_td, supply_td, image_td] 

            # 데이터 프레임 인덱스 증가
            index = index + 1

        # 오류가 발생했을 경우 발생한 페이지 재접속하여 크롤링 시도
        except ElementNotInteractableException:
            print(category_td + " pass")
            Crawling(driver, page, type, url_xpath, df_alchol, i)


        except ElementClickInterceptedException:
            print(category_td + " pass")
            Crawling(driver, page, type, url_xpath, df_alchol, i)


        except NoSuchElementException:
            print( category_td + " NoSuchElementException")
            Crawling(driver, page, type, url_xpath, df_alchol, i)


        except IndexError:
            print( category_td + " IndexError")
            Crawling(driver, page, type, url_xpath, df_alchol, i)


        driver.back()   


현재 1페이지 크롤링 중
현재 2페이지 크롤링 중
현재 3페이지 크롤링 중
현재 4페이지 크롤링 중
현재 5페이지 크롤링 중
현재 6페이지 크롤링 중
현재 7페이지 크롤링 중
전통 소주 증류식 NoSuchElementException


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="__next"]/div/div/div[1]/a[5]/div[1]/img"}
  (Session info: chrome=111.0.5563.65)
Stacktrace:
Backtrace:
	(No symbol) [0x006EDCE3]
	(No symbol) [0x006839D1]
	(No symbol) [0x00594DA8]
	(No symbol) [0x005C019F]
	(No symbol) [0x005C03AB]
	(No symbol) [0x005EEE62]
	(No symbol) [0x005DAF14]
	(No symbol) [0x005ED57C]
	(No symbol) [0x005DACC6]
	(No symbol) [0x005B6F68]
	(No symbol) [0x005B80CD]
	GetHandleVerifier [0x00963832+2506274]
	GetHandleVerifier [0x00999794+2727300]
	GetHandleVerifier [0x0099E36C+2746716]
	GetHandleVerifier [0x00796690+617600]
	(No symbol) [0x0068C712]
	(No symbol) [0x00691FF8]
	(No symbol) [0x006920DB]
	(No symbol) [0x0069C63B]
	BaseThreadInitThunk [0x761500F9+25]
	RtlGetAppContainerNamedObjectPath [0x77157BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77157B8E+238]


In [15]:
df_alchol.to_csv('alcohol_add_data3.csv')

In [19]:
df_alchol

,name,percent,category,produce,supply,img_url
0,경복궁 소주 40,40.0%,전통 소주 증류식,지비지 스피리츠 주식회사 농업회사법인,지비지 스피리츠 주식회사 농업회사법인,https://cdn.veluga.kr/files/supplier/369/drink...
1,경복궁 소주 20,20.0%,전통 소주 증류식,지비지 스피리츠 주식회사 농업회사법인,지비지 스피리츠 주식회사 농업회사법인,https://cdn.veluga.kr/files/supplier/369/drink...
2,산사춘,12.0%,전통 소주,농업회사법인 배상면주가,하이트진로 국산,https://cdn.veluga.kr/drinks/0/main/4345286d6d...
3,숙성증류주 혼,22.0%,전통 소주 증류식,농업회사법인 (주)제이엘,주식회사 골든블루,https://cdn.veluga.kr/files/supplier/90/drinks...
4,농태기,25.0%,전통 소주 증류식,하나도가,부국상사,https://cdn.veluga.kr/files/supplier/42/drinks...
...,...,...,...,...,...,...
119,안동소주 일품 40,40.0%,전통 소주 증류식,안동소주일품(주),한국술유통,https://cdn.veluga.kr/files/supplier/temporary...
120,필 25,25.0%,전통 소주 증류식,술아원,한국술유통,https://cdn.veluga.kr/files/supplier/temporary...
121,여주명주 려40,40.0%,전통 소주 증류식,국순당 여주명주,한국술유통,https://cdn.veluga.kr/files/supplier/temporary...
122,여주명주 려25,25.0%,전통 소주 증류식,국순당 여주명주,한국술유통,https://cdn.veluga.kr/files/supplier/temporary...


In [10]:
df_alchol['category'].unique()

array(['리큐르'], dtype=object)

In [27]:
alchol_df1 = pd.read_csv('alcohol_add_data.csv', index_col= 0)
alchol_df2 = pd.read_csv('alcohol_add_data1.csv', index_col=0)
alchol_df3 = pd.read_csv('alcohol_add_data2.csv', index_col=0)
alchol_df4 = pd.read_csv('alcohol_add_data3.csv', index_col=0)
alchol_df5 = pd.read_csv('alchol_tak.csv', index_col=0)



In [29]:
alchol_df = pd.concat([alchol_df1, alchol_df2,alchol_df3,alchol_df4], ignore_index= True)

In [30]:
alchol_df.to_csv('add_alcohol_df.csv')

In [32]:
alchol_df = pd.read_csv('add_alcohol_df.csv', index_col= 0)

In [33]:
alchol_df

,name,percent,category,produce,supply,img_url
0,레인보우 R,17.0%,리큐르,대한주조,레인보우 주식회사,https://cdn.veluga.kr/files/supplier/385/drink...
1,카페 보르게티,25.0%,리큐르,보르게티,인덜지 (주),https://cdn.veluga.kr/drinks/24085/main/452423...
2,모차르트 초콜릿 다크 프로모션,17.0%,리큐르,MOZART DISTILLLERIE GMBH,롯데칠성음료 스피리츠부문,https://cdn.veluga.kr/drinks/23628/main/c61d2b...
3,모차르트 초콜릿 크림 프로모션,17.0%,리큐르,MOZART DISTILLLERIE GMBH,롯데칠성음료 스피리츠부문,https://cdn.veluga.kr/drinks/23627/main/a664d7...
4,모차르트 초콜릿 다크,17.0%,리큐르,MOZART DISTILLLERIE GMBH,롯데칠성음료 스피리츠부문,https://cdn.veluga.kr/drinks/0/main/b47f7a59bb...
...,...,...,...,...,...,...
393,안동소주 일품 40,40.0%,전통 소주 증류식,안동소주일품(주),한국술유통,https://cdn.veluga.kr/files/supplier/temporary...
394,필 25,25.0%,전통 소주 증류식,술아원,한국술유통,https://cdn.veluga.kr/files/supplier/temporary...
395,여주명주 려40,40.0%,전통 소주 증류식,국순당 여주명주,한국술유통,https://cdn.veluga.kr/files/supplier/temporary...
396,여주명주 려25,25.0%,전통 소주 증류식,국순당 여주명주,한국술유통,https://cdn.veluga.kr/files/supplier/temporary...


In [46]:
alchol_df['category'].unique()

array([nan, '스피릿', '칵테일', '증류식', '사이더', '맥주', '기타', '윗비어', '페일 라거', '라거',
       '스타우트', '사워', '임페리얼 스타우트', '와일드 에일', 'IPA', '필스너', '더블 IPA',
       '페일 에일', '헬레스 라거', '고제', '메르첸', '벨지안 에일', '블론드 에일', '트리펠', '람빅',
       '프루트 비어', '바이젠 복', '바이젠', '위트 비어', '두벨', '세종', '포터', '발틱 포터',
       '벨지안 스트롱 에일', '임페리얼 IPA', '브라운 에일', '베를리너 바이세', '다크 IPA', '세션 IPA',
       '위트 에일', '다크 라거', '골든 에일', '엠버 에일', '쿼드루펠', '라들러', '괴즈',
       '잉글리시 스트롱 에일', '헤페 바이젠', '프루티드 람빅', '올드 에일', '쾰쉬', '아메리칸 스트롱 에일',
       '발리 와인', '아이스 복', '도펠 복', '둥켈', '엠버 라거', '그리셋', '스모크 비어', '둥켈 바이젠',
       '스카치 에일', '희석식', '막걸리'], dtype=object)

In [34]:
alchol_df['class'] = np.nan
alchol_df['category1'] = np.nan

category = alchol_df['category'].str.split(' ')

for i, j in enumerate(category):
    
    if len(j) == 1 :
        alchol_df['class'][i] = j[0]

    if len(j) == 2 :
        if j[0] == '탁주' or j[0] == '과실주' or j[0] == '논알콜':
            alchol_df['class'][i] = j[0]
            alchol_df['category1'][i] = j[1]

        else :
            alchol_df['class'][i] = j[0] + " " +j[1]

    if len(j) == 3 :
        alchol_df['class'][i] = j[0] + " " + j[1]
        alchol_df['category1'][i] = j[2]


    if len(j) == 4 :
        alchol_df['class'][i] = j[0] + " " + j[1]
        alchol_df['category1'][i] = j[2] + " " + j[3]

    if len(j) == 5 :
        alchol_df['class'][i] = j[0] + " " + j[1]
        alchol_df['category1'][i] = j[2] + " " + j[3] + " " + j[4]



C:\Users\5-12\AppData\Local\Temp\ipykernel_21604\3444138487.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alchol_df['class'][i] = j[0]
C:\Users\5-12\AppData\Local\Temp\ipykernel_21604\3444138487.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alchol_df['category1'][i] = j[2]


In [35]:
alchol_df.drop(columns= '580' , inplace= True)

KeyError: "['580'] not found in axis"

In [36]:
alchol_df

,name,percent,category,produce,supply,img_url,class,category1
0,레인보우 R,17.0%,리큐르,대한주조,레인보우 주식회사,https://cdn.veluga.kr/files/supplier/385/drink...,리큐르,NaN
1,카페 보르게티,25.0%,리큐르,보르게티,인덜지 (주),https://cdn.veluga.kr/drinks/24085/main/452423...,리큐르,NaN
2,모차르트 초콜릿 다크 프로모션,17.0%,리큐르,MOZART DISTILLLERIE GMBH,롯데칠성음료 스피리츠부문,https://cdn.veluga.kr/drinks/23628/main/c61d2b...,리큐르,NaN
3,모차르트 초콜릿 크림 프로모션,17.0%,리큐르,MOZART DISTILLLERIE GMBH,롯데칠성음료 스피리츠부문,https://cdn.veluga.kr/drinks/23627/main/a664d7...,리큐르,NaN
4,모차르트 초콜릿 다크,17.0%,리큐르,MOZART DISTILLLERIE GMBH,롯데칠성음료 스피리츠부문,https://cdn.veluga.kr/drinks/0/main/b47f7a59bb...,리큐르,NaN
...,...,...,...,...,...,...,...,...
393,안동소주 일품 40,40.0%,전통 소주 증류식,안동소주일품(주),한국술유통,https://cdn.veluga.kr/files/supplier/temporary...,전통 소주,증류식
394,필 25,25.0%,전통 소주 증류식,술아원,한국술유통,https://cdn.veluga.kr/files/supplier/temporary...,전통 소주,증류식
395,여주명주 려40,40.0%,전통 소주 증류식,국순당 여주명주,한국술유통,https://cdn.veluga.kr/files/supplier/temporary...,전통 소주,증류식
396,여주명주 려25,25.0%,전통 소주 증류식,국순당 여주명주,한국술유통,https://cdn.veluga.kr/files/supplier/temporary...,전통 소주,증류식


In [35]:
alchol_df.to_csv('test.csv')

In [37]:
alchol_df.drop(columns= 'category', inplace= True)

In [38]:
alchol_df = alchol_df[['name', 'percent','class','category1','produce' ,'supply', 'img_url']]

In [39]:
alchol_df = alchol_df.rename(columns={"category1" : "category"})

In [40]:
alchol_df

,name,percent,class,category,produce,supply,img_url
0,레인보우 R,17.0%,리큐르,NaN,대한주조,레인보우 주식회사,https://cdn.veluga.kr/files/supplier/385/drink...
1,카페 보르게티,25.0%,리큐르,NaN,보르게티,인덜지 (주),https://cdn.veluga.kr/drinks/24085/main/452423...
2,모차르트 초콜릿 다크 프로모션,17.0%,리큐르,NaN,MOZART DISTILLLERIE GMBH,롯데칠성음료 스피리츠부문,https://cdn.veluga.kr/drinks/23628/main/c61d2b...
3,모차르트 초콜릿 크림 프로모션,17.0%,리큐르,NaN,MOZART DISTILLLERIE GMBH,롯데칠성음료 스피리츠부문,https://cdn.veluga.kr/drinks/23627/main/a664d7...
4,모차르트 초콜릿 다크,17.0%,리큐르,NaN,MOZART DISTILLLERIE GMBH,롯데칠성음료 스피리츠부문,https://cdn.veluga.kr/drinks/0/main/b47f7a59bb...
...,...,...,...,...,...,...,...
393,안동소주 일품 40,40.0%,전통 소주,증류식,안동소주일품(주),한국술유통,https://cdn.veluga.kr/files/supplier/temporary...
394,필 25,25.0%,전통 소주,증류식,술아원,한국술유통,https://cdn.veluga.kr/files/supplier/temporary...
395,여주명주 려40,40.0%,전통 소주,증류식,국순당 여주명주,한국술유통,https://cdn.veluga.kr/files/supplier/temporary...
396,여주명주 려25,25.0%,전통 소주,증류식,국순당 여주명주,한국술유통,https://cdn.veluga.kr/files/supplier/temporary...


In [41]:
alchol_df.to_csv('add_alcohol_data.csv')